* データマイニング特別演習
* 気象庁からダウンロードしたさいたま市の気温データを分析

In [1]:
# google colab の場合は以下のコメントアウトを外す
#!wget https://github.com/KHiraGit/sudspg_ds/raw/main/dm02_data.zip
#!unzip -d data dm02_data.zip

In [2]:
# 利用するライブラリをインポート
import os
import glob
import re
import csv
import openpyxl

In [3]:
# glob 関数を使って dataフォルダの csvファイルのリストを作成 (年でソート)
files = sorted(glob.glob(os.path.join('data', '*.csv')))

dict = {}
for file in files: # リストのファイルを１つずつ順に処理
    with open(file, encoding='shift-jis') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) == 0 or not re.search(r'^\d\d\d\d', row[0]): #  or not re.search(r'^\d\d\.\d', row[1])
                continue # 空行、データ行以外、欠損値の行はスキップ
            m = re.search(r'^(\d\d\d\d)/(\d+/\d+)', row[0])
            if m is None:
                continue
            year = m.group(1)
            date = m.group(2)
            if re.search(r'\d\d\.\d', row[1]):
                temp = float(row[1])
            else:
                temp = -1.0
            if date not in dict.keys():
                dict[date] = {}
            if year not in dict[date].keys():
                dict[date][year] = 0
            if re.search(r'\d\d\.\d', row[1]) and float(row[1]) >= 35.0: # 35度以上の時間帯を日ごとにカウント
                dict[date][year] = dict[date][year] + 1

# 確認用に 8/1 のデータを表示
print('8/1', dict['8/1'])

8/1 {'2013': 0, '2014': 2, '2015': 5, '2016': 0, '2017': 0, '2018': 4, '2019': 3, '2020': 0, '2021': 1, '2022': 6, '2023': 0, '2024': 0}


In [4]:
wb = openpyxl.Workbook()
ws = wb.active

# header を作成
for key_1 in dict:
    _tmp = ['']
    for key_2 in dict[key_1].keys():
        _tmp.append(key_2)
ws.append(_tmp)

# データ本体を作成
for key_1 in dict:
    _tmp = [key_1]
    for key_2 in dict[key_1].keys():
        _tmp.append(dict[key_1][key_2])
    ws.append(_tmp)

chart1 = openpyxl.chart.BarChart()
chart1.height = 20 # 作成するグラフの高さを設定
chart1.width = 30 # 作成するグラフの幅を設定
chart1.type = "col"
chart1.grouping = "stacked" # 積み上げ棒グラフに設定
chart1.title = "35度以上の時間数" # グラフタイトルの設定
chart1.y_axis.title = '日数' # y軸のラベルを設定
chart1.x_axis.title = '日付' # x軸のラベルを設定
chart1.overlap = 100 # 棒のオーバーラップを設定
# データの範囲、カテゴリーの範囲を設定
data = openpyxl.chart.Reference(ws, min_col=2, min_row=1, max_row=94, max_col=12)
cats = openpyxl.chart.Reference(ws, min_col=1, min_row=2, max_row=94)
chart1.add_data(data, titles_from_data=True)
chart1.set_categories(cats)
chart1.shape = 4
ws.add_chart(chart1, "N2") # グラフの出力先を設定
# 条件付き書式を設定 (時間数が多くなると赤背景で表示)
rule = openpyxl.formatting.rule.ColorScaleRule(
    start_type='min',
    start_color='FFFFFF',
    end_type='max',
    end_color='FF0000'
)
ws.conditional_formatting.add('B2:L94', rule)

wb.save(os.path.join('data', 'jma_data.xlsx'))